In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from tensorflow.python.ops.numpy_ops import np_config
import os

np_config.enable_numpy_behavior()

2023-05-08 17:04:41.850061: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-08 17:04:41.871966: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-08 17:04:41.972175: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-08 17:04:41.972886: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 17:04:42.730220: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
model = tf.keras.models.load_model("/home/vishesh/Desktop/datasets/ships-data/Xception-10-0.74.hdf5", compile=False)
model.compile(optimizer=tfa.optimizers.RectifiedAdam(), loss='categorical_crossentropy')

feature_extractor = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer('dense_1').output)

2023-05-08 17:04:45.825422: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-05-08 17:04:45.825446: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: vishesh-Precision-7560
2023-05-08 17:04:45.825449: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: vishesh-Precision-7560
2023-05-08 17:04:45.825554: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 470.182.3
2023-05-08 17:04:45.825563: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 470.182.3
2023-05-08 17:04:45.825566: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 470.182.3


In [8]:
# input_data_path = "/home/vishesh/Desktop/datasets/ships-data/X_true_val_240_320/1.npy"
images_root = "path/to/images"
input_data_dir = "/home/vishesh/Desktop/datasets/ships-data/X_true_train_240_320"
ground_truth_dir = "/home/vishesh/Desktop/datasets/ships-data/Y_true_train_240_320"

input_data = None
ground_truth = None

for i in range(1,16):
    print(i)
    input_data_path = os.path.join(input_data_dir, f"{i}.npy")
    ground_truth_path = os.path.join(ground_truth_dir, f"{i}.npy")
    
    if i == 1:
        input_data = np.load(input_data_path)
        ground_truth = np.load(ground_truth_path)
        continue
        
        
    new_input_data = np.load(input_data_path)
    new_ground_truth = np.load(ground_truth_path)
    

    input_data = np.vstack((input_data, new_input_data))
    ground_truth = np.hstack((ground_truth, new_ground_truth))
    

        


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [4]:
urls = [f"http://localhost:8081/img{i}.png" for i in range(1, input_data.shape[0]+1)]

In [9]:
preds = np.argmax(model.predict(input_data, batch_size=1), axis=1)

14713/14713 [==============================] - 626s 42ms/step


In [5]:
features = feature_extractor.predict(input_data, batch_size=1)

14713/14713 [==============================] - 623s 42ms/step


In [6]:
data_dict = {'url': urls, 'embeddings':features.tolist(), 'predicted': preds, 'actual': ground_truth}
df = pd.DataFrame(data_dict)

df.head(10)

,url,embeddings,actual
0,http://localhost:8081/img1.png,"[0.0, 0.2656961977481842, 0.0, 0.0, 0.33624574...",3.0
1,http://localhost:8081/img2.png,"[0.0, 1.182692527770996, 0.0, 0.44767528772354...",2.0
2,http://localhost:8081/img3.png,"[0.0, 0.5855140089988708, 0.0, 0.0, 0.0, 0.0, ...",1.0
3,http://localhost:8081/img4.png,"[0.0, 1.0834341049194336, 0.0, 0.4188486635684...",3.0
4,http://localhost:8081/img5.png,"[1.0779553651809692, 0.0, 0.0, 0.0, 0.62473011...",6.0
5,http://localhost:8081/img6.png,"[0.0, 1.1474183797836304, 0.801287055015564, 1...",8.0
6,http://localhost:8081/img7.png,"[0.0, 0.0, 0.3171471655368805, 0.2210593521595...",0.0
7,http://localhost:8081/img8.png,"[0.7096734642982483, 0.0038759075105190277, 0....",6.0
8,http://localhost:8081/img9.png,"[0.0, 0.0, 1.1061588525772095, 0.0, 0.44484469...",4.0
9,http://localhost:8081/img10.png,"[0.12334109097719193, 1.2066916227340698, 0.34...",2.0


In [13]:
df.to_csv('/home/vishesh/Desktop/datasets/ships-data/train.csv') 

In [3]:
df = pd.read_csv('/home/vishesh/Desktop/datasets/ships-data/rando.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

In [10]:
df["predicted"] = preds

In [12]:
df.head(20)

,url,embeddings,actual,predicted
0,http://localhost:8081/img1.png,"[0.0, 0.2656961977481842, 0.0, 0.0, 0.33624574...",3.0,3
1,http://localhost:8081/img2.png,"[0.0, 1.182692527770996, 0.0, 0.44767528772354...",2.0,2
2,http://localhost:8081/img3.png,"[0.0, 0.5855140089988708, 0.0, 0.0, 0.0, 0.0, ...",1.0,1
3,http://localhost:8081/img4.png,"[0.0, 1.0834341049194336, 0.0, 0.4188486635684...",3.0,3
4,http://localhost:8081/img5.png,"[1.0779553651809692, 0.0, 0.0, 0.0, 0.62473011...",6.0,6
5,http://localhost:8081/img6.png,"[0.0, 1.1474183797836304, 0.801287055015564, 1...",8.0,8
6,http://localhost:8081/img7.png,"[0.0, 0.0, 0.3171471655368805, 0.2210593521595...",0.0,0
7,http://localhost:8081/img8.png,"[0.7096734642982483, 0.0038759075105190277, 0....",6.0,6
8,http://localhost:8081/img9.png,"[0.0, 0.0, 1.1061588525772095, 0.0, 0.44484469...",4.0,4
9,http://localhost:8081/img10.png,"[0.12334109097719193, 1.2066916227340698, 0.34...",2.0,2


In [16]:
df.iloc[2000]

url                           http://localhost:8081/img2001.png
embeddings    [0.0, 0.20759457349777222, 0.0, 0.0, 0.1846409...
actual                                                      0.0
predicted                                                     0
Name: 2000, dtype: object